In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !unzip "/content/drive/MyDrive/Datasets.zip" -d "/content/drive/MyDrive/"

Archive:  /content/drive/MyDrive/Datasets.zip
   creating: /content/drive/MyDrive/Datasets/
   creating: /content/drive/MyDrive/Datasets/Test/
   creating: /content/drive/MyDrive/Datasets/Test/hridoy/
  inflating: /content/drive/MyDrive/Datasets/Test/hridoy/1.jpg  
  inflating: /content/drive/MyDrive/Datasets/Test/hridoy/10.jpg  
  inflating: /content/drive/MyDrive/Datasets/Test/hridoy/100.jpg  
  inflating: /content/drive/MyDrive/Datasets/Test/hridoy/11.jpg  
  inflating: /content/drive/MyDrive/Datasets/Test/hridoy/12.jpg  
  inflating: /content/drive/MyDrive/Datasets/Test/hridoy/13.jpg  
  inflating: /content/drive/MyDrive/Datasets/Test/hridoy/14.jpg  
  inflating: /content/drive/MyDrive/Datasets/Test/hridoy/15.jpg  
  inflating: /content/drive/MyDrive/Datasets/Test/hridoy/16.jpg  
  inflating: /content/drive/MyDrive/Datasets/Test/hridoy/17.jpg  
  inflating: /content/drive/MyDrive/Datasets/Test/hridoy/18.jpg  
  inflating: /content/drive/MyDrive/Datasets/Test/hridoy/19.jpg  
  infla

In [9]:
# Deep Learning CNN model to recognize face
'''This script uses a database of images and creates CNN model on top of it to test
   if the given image is recognized correctly or not'''
 
'''####### IMAGE PRE-PROCESSING for TRAINING and TESTING data #######'''
 
# Specifying the folder where images are present
TrainingImagePath='/content/drive/MyDrive/Datasets/Train/'
 
from keras.preprocessing.image import ImageDataGenerator
# Understand more about ImageDataGenerator at below link
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
 
# Defining pre-processing transformations on raw images of training data
# These hyper parameters helps to generate slightly twisted versions
# of the original image, which leads to a better model, since it learns
# on the good and bad mix of images
train_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)
 
# Defining pre-processing transformations on raw images of testing data
# No transformations are done on the testing images
test_datagen = ImageDataGenerator()
 
# Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')
 
 
# Generating the Testing Data
test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')
 
# Printing class labels for each face
test_set.class_indices

Found 800 images belonging to 4 classes.
Found 800 images belonging to 4 classes.


{'hridoy': 0, 'partho': 1, 'shaon': 2, 'zayed': 3}

In [10]:
'''############ Creating lookup table for all faces ############'''
# class_indices have the numeric tag for each face
TrainClasses=training_set.class_indices
 
# Storing the face and the numeric tag for future reference
ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName
 
# Saving the face map for future reference
import pickle
with open("ResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)
 
# The model will give answer as a numeric tag
# This mapping will help to get the corresponding face name for it
print("Mapping of Face and its ID",ResultMap)
 
# The number of neurons for the output layer is equal to the number of faces
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

Mapping of Face and its ID {0: 'hridoy', 1: 'partho', 2: 'shaon', 3: 'zayed'}

 The Number of output neurons:  4


In [12]:
'''######################## Create CNN deep learning model ########################'''
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
 
'''Initializing the Convolutional Neural Network'''
classifier= Sequential()
 
''' STEP--1 Convolution
# Adding the first layer of CNN
# we are using the format (64,64,3) because we are using TensorFlow backend
# It means 3 matrix of size (64X64) pixels representing Red, Green and Blue components of pixels
'''
classifier.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64,64,3), activation='relu'))
 
'''# STEP--2 MAX Pooling'''
classifier.add(MaxPool2D(pool_size=(2,2)))
 
'''############## ADDITIONAL LAYER of CONVOLUTION for better accuracy #################'''
classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
 
classifier.add(MaxPool2D(pool_size=(2,2)))
 
'''# STEP--3 FLattening'''
classifier.add(Flatten())
 
'''# STEP--4 Fully Connected Neural Network'''
classifier.add(Dense(64, activation='relu'))
 
classifier.add(Dense(OutputNeurons, activation='softmax'))
 
'''# Compiling the CNN'''
#classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])
 
###########################################################
import time
# Measuring the time taken by the model to train
StartTime=time.time()
 
# Starting the model training
classifier.fit_generator(
                    training_set,
                    epochs=10,
                    validation_data=test_set,
                    validation_steps=10)
 
EndTime=time.time()
print("###### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
25/25 [==============================] - 14s 527ms/step - loss: 28.9458 - accuracy: 0.6812 - val_loss: 0.0331 - val_accuracy: 0.9875
Epoch 2/10
25/25 [==============================] - 13s 520ms/step - loss: 0.0544 - accuracy: 0.9900 - val_loss: 0.0028 - val_accuracy: 0.9969
Epoch 3/10
25/25 [==============================] - 13s 521ms/step - loss: 0.0230 - accuracy: 0.9925 - val_loss: 0.0310 - val_accuracy: 0.9969
Epoch 4/10
25/25 [==============================] - 13s 521ms/step - loss: 0.0201 - accuracy: 0.9962 - val_loss: 0.0035 - val_accuracy: 0.9969
Epoch 5/10
25/25 [==============================] - 13s 535ms/step - loss: 0.0083 - accuracy: 0.9962 - val_loss: 5.5648e-04 - val_accuracy: 1.0000
Epoch 6/10
25/25 [==============================] - 14s 537ms/step - loss: 0.0284 - accuracy: 0.9912 - val_loss: 0.0038 - val_accuracy: 1.0000
Epoch 7/10
25/25 [==============================] - 13s 535ms/step - loss: 0.0984 - accuracy: 0.9750 - val_loss: 0.0115 - val_accuracy: 0

In [13]:
'''########### Making single predictions ###########'''
import numpy as np
from keras.preprocessing import image
 
ImagePath='/content/drive/MyDrive/Datasets/Test/partho/10.jpg'
test_image=image.load_img(ImagePath,target_size=(64, 64))
test_image=image.img_to_array(test_image)
 
test_image=np.expand_dims(test_image,axis=0)
 
result=classifier.predict(test_image,verbose=0)
#print(training_set.class_indices)
 
print('####'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])

########################################
Prediction is:  partho
